In [2]:
import numpy as np
import os
import json, time, copy
import math

import random
import pickle

from word2number import w2n
import string, re
from collections import Counter, defaultdict
from pprint import pprint
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner","textcat","parser"])

In [3]:
def detectNum(l):
    result = []
    for w in l:
        try: result.append(str(int(w)))
        except: pass
    return result
def toNum(word):
    if word == 'point': return word
    try: return w2n.word_to_num(word)
    except:
        return word

def normalize_text(s):
    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text): # additional: converting numbers to digit form
        return " ".join([str(toNum(w)) for w in text.split()])

    def remove_punc(text):
        exclude = set(string.punctuation) - set(['.'])
        text1 = "".join(ch for ch in text if ch not in exclude)
        return re.sub(r"\.(?!\d)", "", text1) # remove '.' if it's not a decimal point

    def lower(text):
        return text.lower()
    
    def lemmatization(text):
        return " ".join([token.lemma_ for token in nlp(text)])

    if len(s.strip()) == 1:
        # accept article and punc if input is a single char
        return white_space_fix(lower(s))
    elif len(s.strip().split()) == 1: 
        # accept article if input is a single word
        return lemmatization(white_space_fix(remove_punc(lower(s))))

    return lemmatization(white_space_fix(remove_articles(remove_punc(lower(s)))))


In [4]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0823_clean_te.json", "r"))
img_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/img_dataset_0823_clean_te.json", "r"))

print(Counter([txt_dataset[k]['split'] for k in txt_dataset]))
print(len(set([txt_dataset[k]['Guid'] for k in txt_dataset])))

print(Counter([img_dataset[k]['split'] for k in img_dataset]))
print(Counter([img_dataset[k]['Qcate'] for k in img_dataset]))
print(Counter([img_dataset[k]['Qcate'] for k in img_dataset if img_dataset[k]['split'] == 'ood_test']))
print(len(set([img_dataset[k]['Guid'] for k in img_dataset])))

Counter({'train': 17812, 'test': 4076, 'val': 2455})
24343
Counter({'train': 16448, 'test': 3464, 'val': 2511})
Counter({'YesNo': 7430, 'Others': 5823, 'choose': 4693, 'number': 2084, 'color': 1832, 'shape': 561})
Counter()
22423


In [5]:
x = []
for k in txt_dataset:
    if txt_dataset[k]['split'] == 'test':
        x.append(len(txt_dataset[k]['A']))
print(Counter(x))
x = []
for k in img_dataset:
    if 'test' in img_dataset[k]['split']:
        x.append(len(img_dataset[k]['A']))
print(Counter(x))

Counter({6: 1366, 5: 1343, 4: 874, 3: 493})
Counter({6: 1579, 5: 780, 4: 703, 3: 402})


In [ ]:
## 同一个sample单独，最后取avg/max，RE<0.3的 full sentence 忽略
### Double check on the clean_te version
eval_f = Evaluate()
bleu4_img_clean = {}
RE_img_clean = {}
F1_img_clean = {}
mul_img_clean = {}

drop = defaultdict(int)
for k in img_dataset:
    if not 'test' in img_dataset[k]['split']: continue
    bleu4_img_clean[k] = []
    RE_img_clean[k] = []
    mul_img_clean[k] = []
    F1_img_clean[k] = []
    datum = img_dataset[k]
    Keywords_A = datum['Keywords_A'].replace('"', "")
    all_A = [a.replace('"', "") for a in datum['A']]
    Qcate = img_dataset[k]['Qcate']
    for i in range(len(all_A)):
        Q = datum['Q'].replace('"', "")
        C = [all_A[i]]
        A_list = all_A[:i] + all_A[i+1:]
        scores = eval_f.evaluate(cand=[C], ref=[A_list], return_scores=True)
        #print(scores)
        
        if Qcate == 'color': F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics(C, Keywords_A, "", color_set)
        elif Qcate == 'shape': F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics(C, Keywords_A, "", shape_set)
        elif Qcate == 'YesNo': F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics(C, Keywords_A, "", yesno_set)
        elif Qcate == 'number': F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics(C, Keywords_A, "", {"NUMBER"})
        else: F1_avg, F1_max, EM, RE_avg, PR_avg = compute_vqa_metrics(C, Keywords_A)
        bleu4_img_clean[k].append(scores['Bleu_4'])
        RE_img_clean[k].append(RE_avg)
        F1_img_clean[k].append(F1_avg)
        if Qcate in ['choose', 'Others']: mul_img_clean[k].append(RE_avg * scores['Bleu_4'])
        else: mul_img_clean[k].append(F1_avg * scores['Bleu_4'])
        
    if len(RE_img_clean) % 500 == 499: print(len(RE_img_clean))
assert len(RE_img_clean) == len(mul_img_clean) == len(bleu4_img_clean) == len(F1_img_clean)
print(len(RE_img_clean))
print(drop)
print(np.sum(list(drop.values())))

In [16]:
from datasets import load_metric
metric = load_metric("bertscore")
def compute_bertscore(cands, a, model_type):
    metric.add_batch(predictions = cands, references = [a]*len(cands))
    score = metric.compute(model_type=model_type)
    return np.mean(score['f1']), np.max(score['f1'])

In [17]:
cands = ["Beijing"]
a = ["Yuzhny, which is an urban locality, is in the same district as Rytkuchi."]
print(np.exp(bart_scorer.score(cands, a)))
print(np.exp(bart_scorer.score(cands, a)))
print(np.exp(bart_scorer.score(a, a)))
print(compute_bertscore(cands, a, "microsoft/deberta-xlarge-mnli"))

[0.00951297]
[0.00951297]
[0.77009736]
(0.4098593592643738, 0.4098593592643738)


In [18]:
cands = ["Both Ireland and England"]
a = "The Gilbert River that is a tributary of the Multnomah Channel on Sauvie Island shares its name with the Gilbert River that is a freshwater tributary of the Cyriac River."
print(np.exp(bart_scorer.score(cands, [a])))
print(np.exp(bart_scorer.score([a], cands)))
print(np.exp(bart_scorer.score([a], [a])))
print(compute_bertscore(cands, a, "microsoft/deberta-xlarge-mnli"))

[0.02257406]
[0.00128582]
[0.82900294]
(0.37695783376693726, 0.37695783376693726)


In [19]:
cands = ["It was made before"] # Correct but too short
a = "The Gathering directed by Randal Kleiser was made before the Dutch rock band The Gathering had formed."
print(np.exp(bart_scorer.score(cands, [a])))
print(np.exp(bart_scorer.score([a], cands)))
print(np.exp(bart_scorer.score([a], [a])))
print(compute_bertscore(cands, a, "microsoft/deberta-xlarge-mnli"))

[0.01384408]
[0.03432885]
[0.72917158]
(0.5283754467964172, 0.5283754467964172)


In [82]:
# Another human reference
cands = ["The movie The Gathering directed by Randal Kleiser was made before the Dutch rock band The Gathering had formed"]
a = "The Gathering directed by Randal Kleiser was made before the Dutch rock band The Gathering had formed."
print(np.exp(bart_scorer.score(cands, [a])))
print(np.exp(bart_scorer.score([a], cands)))
print(compute_bertscore(cands, a, "microsoft/deberta-xlarge-mnli"))

[0.55254496]
[0.22784942]
(0.9338558912277222, 0.9338558912277222)


In [84]:
cands = ["before"]
a = "The Gathering directed by Randal Kleiser was made before the Dutch rock band The Gathering had formed."
print(np.exp(bart_scorer.score(cands, [a])))
print(np.exp(bart_scorer.score([a], cands)))
print(compute_bertscore(cands, a, "microsoft/deberta-xlarge-mnli"))

[0.00817325]
[0.00033844]
(0.4296928644180298, 0.4296928644180298)


In [85]:
# Same sentence, disordered
cands = ["Dutch The by formed directed band before made was Randalthe rock Kleiser had The Gathering Gathering."]
a = "The Gathering directed by Randal Kleiser was made before the Dutch rock band The Gathering had formed."
print(np.exp(bart_scorer.score(cands, [a])))
print(np.exp(bart_scorer.score([a], cands)))
print(compute_bertscore(cands, a, "microsoft/deberta-xlarge-mnli"))

[0.05339534]
[0.0007564]
(0.5756980776786804, 0.5756980776786804)


In [86]:
# An irrelevant candidate
cands = ["Both basketball and football"]
a = "The Gathering directed by Randal Kleiser was made before the Dutch rock band The Gathering had formed."
print(np.exp(bart_scorer.score(cands, [a])))
print(np.exp(bart_scorer.score([a], cands)))
print(compute_bertscore(cands, a, "microsoft/deberta-xlarge-mnli"))

[0.00566049]
[0.00252041]
(0.39895495772361755, 0.39895495772361755)


In [87]:
cands = ['Yamata No Orochi']
a = "Vampires have more power than Yamata No Orochi"
print(np.exp(bart_scorer.score(cands, [a])))
print(np.exp(bart_scorer.score([a], cands)))
print(compute_bertscore(cands, a, "microsoft/deberta-xlarge-mnli"))

[0.01736348]
[0.12752726]
(0.6037829518318176, 0.6037829518318176)


In [8]:
import sys
sys.path.append("/home/yingshac/CYS/WebQnA/VLP/BARTScore")

In [9]:
from bart_score import BARTScorer

In [10]:
bart_scorer = BARTScorer(device='cuda:0', checkpoint='facebook/bart-large-cnn')

In [8]:
bart_scorer.score(['This is interesting.'], ['This is fun.'])

AttributeError: 'tuple' object has no attribute 'logits'